In [1]:
import polars as pl

In [18]:
data1 = pl.read_excel(source='data/data1.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data2 = pl.read_excel(source='data/data2.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data3 = pl.read_excel(source='data/data3.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data4 = pl.read_excel(source='data/data4.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data5 = pl.read_excel(source='data/data5.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")

Оставляем только те строки, где известен победитель и есть ИНН заказчика и поставщика.

In [19]:
fdata1 = data1.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata2 = data2.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata3 = data3.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata4 = data4.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata5 = data5.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())

In [146]:
data = pl.concat([fdata1, fdata2, fdata3, fdata5])
data = data.with_columns(pl.col("ИНН поставщика").str.to_integer())
data = pl.concat([data, fdata4])
data.shape

(16240, 18)

In [147]:
data = data.rename({'Дата \nокончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ':'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                    'Дата начала подачи заявок/Дата начала исполнения контракта / Дата публикации ППГ' : 'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                    'Дата  окончания проведения торгов' : 'Дата окончания проведения торгов'})

Оставляем только те строки, где известно снижение на торгах и статус допуска допущен

In [148]:
data = data.filter((pl.col("Статус допуска")==pl.lit("Допущен")) & pl.col("Снижение на торгах,%").is_not_null())
data.shape

(14765, 18)

Уберем нереалистичные значения в ставках в Снижение на торгах,% 

In [151]:
data = data.filter(pl.col('Снижение на торгах,%')>=-100)
data.shape

(14739, 18)

Теперь работаем с датами

In [152]:
ddata = data.with_columns(pl.col('Дата публикации').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания проведения торгов').str.to_date(format="%m-%d-%y", strict=False))

In [156]:
data = ddata.filter(pl.col('Дата публикации').is_not_null() & 
                         pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').is_not_null() & 
                         pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').is_not_null() & 
                         pl.col('Дата окончания проведения торгов').is_not_null())
data.shape

(11460, 18)

In [159]:
data.head(1)

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Сфера деятельности,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,"Снижение на торгах,%",Форма публикации,Тип торгов
i64,str,i64,f64,str,str,str,date,date,date,date,str,i64,str,str,f64,str,str
1,"""МУП ""РЕСУРС""""",6614001840,151972.23,"""[ОКПД2 28.14] Арматура трубопр…","""Свердловская область""","""Свердловская область""",2023-05-30,2023-05-31,2023-05-30,2023-05-30,"""ООО ""САНТЕХКОМПЛЕКТ-ТАГИЛ""""",6623019699,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…"


In [162]:
data.write_excel( dtype_formats={pl.Date: "yyyy-mm-dd"})

Добавляем реестр недобросовестных поставщиков

In [168]:
notrust = pl.read_json('notrust_list/res.json')
notrust

suplier_name,suplier_inn,no_trust_now,no_trust_before
str,str,i64,i64
"""ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС…","""7203489478""",0,1
"""ООО ЧОО «ГРОМ»""","""7718970047""",0,1
"""ООО ЧОО «ГРОМ»""","""7718970047""",0,1
"""НЕДАШКОВСКИЙ АРТЕМ АНАТОЛЬЕВИЧ""","""420523840689""",0,1
"""ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС…","""5404071052""",0,1
…,…,…,…
"""ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС…","""9701073162""",0,1
"""общество с ограниченной ответс…","""7606112298""",0,1
"""ИП Пименов Т.П.""","""772479558684""",0,1


In [169]:
notrust = notrust.rename({'suplier_name':'Поставщик', 'suplier_inn':'ИНН поставщика', 'no_trust_now':'РНП сейчас', 'no_trust_before':'РНП ранее'})
notrust = notrust.with_columns(pl.col('ИНН поставщика').str.to_integer())
notrust.head(1)

Поставщик,ИНН поставщика,РНП сейчас,РНП ранее
str,i64,i64,i64
"""ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС…",7203489478,0,1


In [171]:
datar = data.join(other=notrust, on='ИНН поставщика', how='left')
datar

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Сфера деятельности,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,"Снижение на торгах,%",Форма публикации,Тип торгов,Поставщик_right,РНП сейчас,РНП ранее
i64,str,i64,f64,str,str,str,date,date,date,date,str,i64,str,str,f64,str,str,str,i64,i64
1,"""МУП ""РЕСУРС""""",6614001840,151972.23,"""[ОКПД2 28.14] Арматура трубопр…","""Свердловская область""","""Свердловская область""",2023-05-30,2023-05-31,2023-05-30,2023-05-30,"""ООО ""САНТЕХКОМПЛЕКТ-ТАГИЛ""""",6623019699,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…",null,null,null
1,"""ФГАОУ ДПО ""АКАДЕМИЯ МИНПРОСВЕЩ…",7718084063,419200.0,"""[ОКПД2 26.20] Компьютеры и пер…","""Москва""","""Москва""",2023-05-31,2023-05-31,2023-05-31,2023-05-31,"""ООО ""АКСОН""""",9723096364,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…",null,null,null
1,"""ФГАОУ ДПО ""АКАДЕМИЯ МИНПРОСВЕЩ…",7718084063,498200.0,"""[ОКПД2 63.11] Услуги по обрабо…","""Москва""","""Москва""",2023-05-31,2023-05-31,2023-05-31,2023-05-31,"""ООО ""ЛАБОРАТОРИЯ ЗАЩИТЫ ИНФОРМ…",7720346630,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…",null,null,null
2,"""АО ""ГГЭС""""",2625032987,null,"""[ОКПД2 46.69] Услуги по оптово…","""Ставропольский край""","""Ставропольский край""",2023-05-24,2023-05-31,2023-05-24,2023-05-31,"""ООО ""ЭКСПЕРТ-ЭЛЕКТРО""""",3023017366,"""Победитель""","""Допущен""",0.0009,"""Торговая процедура""",null,null,null,null
1,"""ФГАОУ ДПО ""АКАДЕМИЯ МИНПРОСВЕЩ…",7718084063,246959.0,"""[ОКПД2 71.20] Услуги в области…","""Санкт-Петербург""","""Санкт-Петербург""",2023-05-31,2023-05-31,2023-05-31,2023-05-31,"""ФГБОУ ВО РГПУ ИМ. А. И. ГЕРЦЕН…",7808027849,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,"""МУП АГО ""АНГАРСКИЙ ВОДОКАНАЛ""""",3801006828,4.6278e6,"""[ОКПД2 42.21] Сооружения и стр…","""Иркутская область""","""Иркутская область""",2024-05-24,2024-06-03,2024-05-23,2024-06-04,"""Драгунов Николай Александрович""",380101928348,"""Победитель""","""Допущен""",0.166782,"""Торговая процедура""","""Запрос цен""",null,null,null
1,"""АО ""ГУБЕРНСКИЕ АПТЕКИ""""",2466189073,1.89e6,"""[ОКПД2 21.20] Препараты лекарс…","""Красноярский край""","""Красноярск""",2024-05-27,2024-06-03,2024-05-27,2024-06-03,"""ООО ""ФАРМЛАЙН""""",7714511476,"""Победитель""","""Допущен""",0.000003,"""Торговая процедура""","""Запрос цен""",null,null,null
1,"""АО ""ГУБЕРНСКИЕ АПТЕКИ""""",2466189073,1.3969e7,"""[ОКПД2 21.20] Препараты лекарс…","""Красноярский край""","""Красноярск""",2024-05-27,2024-06-03,2024-05-27,2024-06-03,"""ООО ""ПУЛЬСАР""""",5074072016,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Запрос цен""",null,null,null


In [177]:
datar = datar.with_columns(pl.when(pl.col('РНП сейчас').is_null()).then(0).alias('РНП сейчас'), pl.when(pl.col('РНП ранее').is_null()).then(0).alias('РНП ранее'))

In [179]:
datar.filter(pl.col('РНП сейчас')==1)

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Сфера деятельности,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,"Снижение на торгах,%",Форма публикации,Тип торгов,Поставщик_right,РНП сейчас,РНП ранее
i64,str,i64,f64,str,str,str,date,date,date,date,str,i64,str,str,f64,str,str,str,i32,i32
